### Comments

- when stopping, the last upload link might need to be added mannually to the csv file
- attention au 'pending' status

# Upload Notebook
Uploads all files to Nakala

## 0. Initialization

In [1]:
## install required packages

# create requirements.txt
! echo "pandas==1.1.4" > requirements_upload.txt
! echo "requests==2.27.1" >> requirements_upload.txt

# install requirements
! pip install -r requirements_upload.txt

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os, ast
import csv, requests, json, time
import pandas as pd
from datetime import datetime

sys.path.insert(0, os.path.abspath('./nakalapyconnect/'))
import nklPushCorpus as npc

In [48]:
data_pd = pd.read_csv('photos_info_nkl.csv')

n_photos = len(data_pd)
n_already_uploaded = sum(data_pd.Uploaded)
n_to_upload = n_photos - n_already_uploaded

print("{} photos have been identified: ".format(n_photos))
print("- {} photos have already been uploaded".format(n_already_uploaded))
print("- {} photos need to be uploaded".format(n_to_upload))

# Endpoint & Clef d'API
apiUrl = npc.myConst.API_URL
apiKey = npc.API_KEY_NKL

print('\nKEYS:')
print('URL:',apiUrl)
print('KEY:',apiKey,'\n')

if apiUrl == 'https://apitest.nakala.fr':
    is_api_test = True
    print('You are working on the TEST API.')
else:
    is_api_test = False
    print("/!\ YOU ARE WORKING ON THE MAIN API, ANY CHANGE IS PERMANENT!")
    
if is_api_test:
    link_base = "https://test.nakala.fr/collection/"
else:
    link_base = "https://nakala.fr/collection/"

342 photos have been identified: 
- 0 photos have already been uploaded
- 342 photos need to be uploaded

KEYS:
URL: https://apitest.nakala.fr
KEY: aae99aba-476e-4ff2-2886-0aaf1bfa6fd2 

You are working on the TEST API.


## 1. Upload collection

### 1.1. Check if already uploaded collections

In [5]:
# Create collection file
list_collections = list(set(','.join(list(data_pd['Collections'])).split(',')))
list_collections

collections = pd.DataFrame(index=list_collections, columns = ['uploaded', 'link', 'error', 'date']).reset_index()
collections = collections.rename({'index': 'collection'},axis=1)
collections['uploaded']=0
collections = collections.sort_values(by='collection').reset_index(drop=True)

collections = collections#.iloc[0:1]
print("{} collections have been found in the photos file".format(len(collections)))

22 collections have been found in the photos file


In [6]:
# find an existing version of collections.csv

name_old_collection_file = 'collections.csv'

try:
    collections_old = pd.read_csv(name_old_collection_file)
    print("an existing collecion file has been found, containing {} collections".format(len(collections_old)))
except:
    collections_old = pd.DataFrame()
    print("/!\ no existing collecion file named '{}' has been found!".format(name_old_collection_file))
    print("If you already uploaded collections, make sure the file hasn't been moved or renamed")

an existing collecion file has been found, containing 22 collections


In [7]:
# isolate new collections

if len(collections_old) > 0:
    merged = pd.merge(collections_old["collection"], collections, on='collection', how='outer', indicator=True)
    collections_new = merged[merged._merge == 'left_only']
    collections_new = collections_new.drop("_merge", axis=1)
    
else:
    collections_new = collections

print("{} new collections have been found".format(len(collections_new)))

0 new collections have been found


In [8]:
# append new collections 
collections_updated = collections_old.append(collections_new)

print("The new file contains {} new collections, including {} existing collections and {} new collections"\
      .format(collections_updated.shape[0],
              collections_old.shape[0],
              collections_new.shape[0]))

print("{} collections have already been uploaded".format(sum(collections_updated['uploaded'] == 1)))

The new file contains 22 new collections, including 22 existing collections and 0 new collections
22 collections have already been uploaded


In [9]:
collections = collections_updated.set_index('collection')
collections

,uploaded,link,error,date
collection,,,,
Chortens,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:29
Fort,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:30
Fortified settlement,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:30
Monastery,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:31
Painted Chorten,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:32
Petroglyphs,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:33
Temple,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:33
Temple Ruin,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:34
f126,1,https://test.nakala.fr/collection/10.34847/nkl...,NaN,2022-12-20 17:46:34


In [10]:
collections.to_csv('collections.csv')

### 1.2. Upload collections not previously uploaded

In [12]:
dict_collection_handles = {}

for collection in collections.index:
    
    if collections.loc[collection, 'uploaded'] == 0:
    
        # création d'un dictionnaire dicoBody contenant le status, un nouveau title et la lang désirée
        dicoBody = {}
        dicoBody["status"] = "public"
        dicoBody["metas"] = []
        metaTitle={"value": collection,
          "lang": "fr",
          "typeUri": "http://www.w3.org/2001/XMLSchema#string",
          "propertyUri": "http://nakala.fr/terms#title"
          }
        dicoBody["metas"].append(metaTitle)

        # appel de la fonction post_collection
        response = npc.post_collections(dicoBody)

        if response.status_code == 201:
            response = json.loads(response.text)
            collections.loc[collection, 'uploaded'] = 1
            collections.loc[collection, 'link'] = link_base+response['payload']['id']
            collections.loc[collection, 'date'] = str(datetime.now())[0:19]
            print("Collection '{}' successfully uploaded: {}".format(collection,
                                                                     collections.loc[collection, 'link']))

        else:
            collections.loc[collection, 'error'] = str(response.text)
            print("Collection {} not uploaded! EROR: {}".format(collection,
                                                                response))

        collections.to_csv('collections.csv')

In [18]:
n_not_uploaded_collections = sum(collections.uploaded == 0)

if n_not_uploaded_collections == 0:
    print("All collections have been uploaded! Proceed to next step.")
else:
    print("/!\ {} collections have not been uploade! Rerun previous cell.".format(n_not_uploaded_collections))

All collections have been uploaded! Proceed to next step.


## 2. Upload photos

In [52]:
# Gather collection handles
for collection in collections.index:
    if collections.loc[collection, 'uploaded'] ==1:
        dict_collection_handles[collection] = collections.loc[collection, 'link'].replace(link_base, '')

In [79]:
# Mapping CSV vers API (on aurait aussi pu remplacer directement ces valeurs dans le CSV)
nakalaTypeDict = {
    "Article de journal": "http://purl.org/coar/resource_type/c_6501",
    "Cours": "http://purl.org/coar/resource_type/c_e059",
    "Image": "http://purl.org/coar/resource_type/c_c513",
    "Poster": "http://purl.org/coar/resource_type/c_6670",
    "Présentation": "http://purl.org/coar/resource_type/c_c94f",
    "Set de données": "http://purl.org/coar/resource_type/c_ddb1",
    "Texte": "http://purl.org/coar/resource_type/c_18cf"
}

nakalaCollectionDict = dict_collection_handles

# 1.1 Lecture du fichier CSV
with open('photos_info_nkl.csv', newline='') as f:
    reader = csv.reader(f)
    dataset = list(reader)
dataset.pop(0) # suppression des titres des colonnes

# 1.2 Préparation d'un fichier de sortie
output = open('output.csv', 'w') # ouverture d'un fichier en mode écriture
outputWriter = csv.writer(output) # création d'un objet pour écrire dans ce fichier
header = ['identifier', 'files', 'title', 'status', 'response'] # nom des colonnes à insérer dans ce fichier
outputWriter.writerow(header) # écriture du nom des colonnes dans ce fichier

# 2. Parcours des différentes lignes du fichier
for num, data in enumerate(dataset):
    
    status = data[0]
    title = data[1]
    filenames = data[2].split(';')
    description = data[3]
    authors = list(filter(None,data[4].split(';')))
    date = data[5]
    keywords = list(filter(None,data[6].split(',')))
    datatype = data[7]
    license = data[8]
    nakalaCollections = list(filter(None,data[9].split(',')))
    is_uploaded = data[11]
        
    if is_uploaded == '0': # on n'upload que si pas déjà uploadé

        # 2.1. Récupération des infos disponibles sur la donnée à créer



        print('///',keywords)

        outputData = ['','',title,'','']; # variable où seront stockées les informations à écrire dans le fichier de sortie

        print('CREATION DE LA DONNEE ' + str(num) + " : " + title)

        # 2.2. Envoi des fichiers à l'API
        files = [] # variable pour stocker les informations retournées en JSON par l'API à chaque upload.
        outputFiles = [] # variable pour stocker les informations à écrire dans le fichier de sortie concernant les fichiers uploadés
        for filename in filenames: # on parcours l'ensemble des fichiers d'une donnée

            print('Envoi du fichier ' + filename + '...') # on affiche un message pour le suivi de l'upload
            goToNextData = False
            # écriture de la requête à l'API (ne contient pas de body en JSON, mais un fichier et un clef d'API)
            payload={}
            postfiles=[('file',(filename,open(filename, 'rb')))]
            headers = {'X-API-KEY': apiKey }
            # appel à l'API pour uploader le fichier
            response = requests.request("POST", apiUrl + '/datas/uploads', headers=headers, data=payload, files=postfiles)
            # si l'upload s'est bien passé, on stocke les informations retournés par l'API dans la variable 'files'
            if ( 201 == response.status_code ):
                # avant de stocker les informations retournées par l'API sur le fichier, on y ajoute une date d'embargo
                file = json.loads(response.text)
                file["embargoed"] = time.strftime("%Y-%m-%d") # on renseigne la date du jour si pas de date d'embargo renseignée
                files.append(file)
                # ajout du nom du fichier et du sha1 aux informations à écrire dans le fichier de sortie
                outputFiles.append(filename + ',' + file["sha1"])
            else:
                # une erreur s'est produite avec un upload
                outputFiles.append(filename) # on ajoute le nom du fichier aux informations à écrire dans le fichier de sortie
                outputData[1] = ';'.join(outputFiles)
                outputData[3] = 'ERROR' # on complète les informations à écrire dans le fichier de sortie en indiquant qu'il y a eu une erreur
                outputData[4] = response.text # on insère ici le retour de l'API
                outputWriter.writerow(outputData) # on écrit dans le fichier de sortie
                print ("Certains fichiers n'ont pas pu être envoyés, on passe à la donnée suivante...") # on affiche un message d'erreur
                goToNextData = True # on stocke dans cette variable qu'il y a eu un problème
                break # on arrête l'upload des autres fichiers de la donnée
        if goToNextData: continue # on passe à la donnée suivante si

        # 2.3. On garde une trace pour le fichier de sortie de la liste des fichiers uploadés
        outputData[1] = ';'.join(outputFiles)

        # 2.4. Reconstruction des métadonnées
        metas = [] # on stocke dans cette variable l'ensemble des métadonnées dans le format attendu

        # la métadonnée type (obligatoire)
        metaType = {
            "value": nakalaTypeDict[datatype], # on insère ici le contenu de la colonne "datatype" mappé sur l'URI correspondante (un seul type par donnnée)
            "typeUri": "http://www.w3.org/2001/XMLSchema#anyURI", # on indique ici que la valeur renseignée est de type URI
            "propertyUri": "http://nakala.fr/terms#type" # on indique ici le champ "type" issu du vocabulaire NAKALA
        }
        metas.append(metaType) # ajout de la métadonnée dans le tableau "metas"

        # la métadonnée titre (obligatoire)
        metaTitle = {
            "value": title, # on insère ici le contenu de la colonne "nakala:title (fr)" (un seul titre fr par donnnée)
            "lang": "fr", # on indique ici la langue du titre (cf. ISO-639-1 ou ISO-639-3 pour les langues moins courantes)
            "typeUri": "http://www.w3.org/2001/XMLSchema#string", # on indique ici que la valeur renseignée est une chaîne de caractères
            "propertyUri": "http://nakala.fr/terms#title" # on indique ici le champ "title" issu du vocabulaire de NAKALA
        }
        metas.append(metaTitle)

        # les métadonnées auteurs (obligatoire pour une donnée publiée)
        if not authors: # si vide, on ajoute une métadonnée nakala:creator "anonyme"
           metaAuthor = {
              "value": None, # sic. None sans guillemet veut dire "null" en Python
              "propertyUri": "http://nakala.fr/terms#creator" # on indique ici le champ "creator" issu du vocabulaire de NAKALA
           }
           metas.append(metaAuthor)
        else:    
            for author in authors: # la colonne "nakala:creator" peut comporter plusieurs valeurs qu'on parcours une à une
                   # pour chaque valeur, on sépare le nom et le prénom pour construire la métadonnée nakala:creator
                surnameGivennameORCID = author.split(',')
                metaAuthor = {
                    "value": { # la valeur de cette métadonnée n'est pas une simple chaîne de caractères, mais un objet composé d'au minimum deux propriétées (givenname et surname)
                        "givenname": surnameGivennameORCID[1], # on insère ici le prénom
                        "surname": surnameGivennameORCID[0] # on insère ici le nom de famille
                    },
                    "propertyUri": "http://nakala.fr/terms#creator" # on indique ici le champ "creator" issu du vocabulaire de NAKALA
                }
                # on ajoute le numéro ORCID (si présent)
                if len(surnameGivennameORCID) == 3:
                    metaAuthor["value"]["orcid"] = surnameGivennameORCID[2] # on insère ici le numéro ORCID
                # ajout de la métadonnée
                metas.append(metaAuthor)

        # la métadonnée date de création (obligatoire pour une donnée publiée)
        if not date: # si vide, on ajoute une métadonnée "nakala:created" "inconnue"
            metaCreated = {
                "value": None,
                "propertyUri": "http://nakala.fr/terms#created"
            }
        else:
            metaCreated = {
                "value": date, # on insère ici le contenu de la colonne "nakala:created" (une seule date de création par donnée)
                "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                "propertyUri": "http://nakala.fr/terms#created"
            }
        metas.append(metaCreated)

        # la métadonnée licence (obligatoire pour une donnée publiée)
        metaLicense = {
            "value": license, # On insère ici le contenu de la colonne "nakala:license" (une seule licence par donnée)
            "typeUri": "http://www.w3.org/2001/XMLSchema#string",
            "propertyUri": "http://nakala.fr/terms#license"
        }
        metas.append(metaLicense)

        # la métadonnée description (facultative)
        metaDescription = {
            "value": description, # on insère ici le contenu de la colonne "dcterms:description (fr)" (une seule description par donnée)
            "lang": "fr",
            "typeUri": "http://www.w3.org/2001/XMLSchema#string",
            "propertyUri": "http://purl.org/dc/terms/description" # notez qu'il ne s'agit plus ici d'une propriété issue du vocabulaire NAKALA, mais du vocabulaire Dcterms
        }
        metas.append(metaDescription)

        # les métadonnées mots-clés (facultatives)
        for keyword in keywords: # on parcours les valeurs de la colonne "dcterms:subject (fr)"
            metaKeyword = {
                "value": keyword, # on insère ici la valeur d'un mot-clé
                "lang": "fr",
                "typeUri": "http://www.w3.org/2001/XMLSchema#string",
                "propertyUri": "http://purl.org/dc/terms/subject" # notez qu'il ne s'agit plus ici d'une propriété issue du vocabulaire NAKALA, mais du vocabulaire Dcterms
            }
            metas.append(metaKeyword)

        #print(metas, '======')

        """
            # 2.5. Reconstruction des droits
            rights = [] # on stocke les droits d'une donnée dans un tableau
            for dataright in datarights: # on parcours les valeurs de la colonne "rights"
                datarightSplit = dataright.split(',') # chaque valeur est composée d'un id et d'un rôle séparé par une virgule. On sépare ces valeurs
                if len(datarightSplit) == 2: # on vérifie qu'on a bien deux valeurs uniquement
                    right = { # on reconstruit l'objet "right" composé des propriétés "id" et "role"
                        "id": nakalaGroupDict[datarightSplit[0]], #on mappe la valeur du CSV avec l'id
                        "role": datarightSplit[1]
                    }
                    rights.append(right) # on ajout l'objet "right" dans le tableau "rights"
        """

        # 2.6. Reconstruction des collections
        collectionsIds = []
        for nakalaCollection in nakalaCollections:
            collectionsIds.append(nakalaCollectionDict[nakalaCollection])

        # 3. Envoi de la donnée à NAKALA
        postdata = { # variable contenant le contenu de la requête
            "status" : "published", # on publie directement les données
            "files" : files,
            "metas" : metas,
            #"rights": rights,
            "collectionsIds": collectionsIds
        }
        content = json.dumps(postdata) # serialisation du contenu en JSON
        headers = { # header de la requête
          'Content-Type': 'application/json',
          'X-API-KEY': apiKey,
        }
        response = requests.request("POST", apiUrl + '/datas', headers=headers, data=content) # requête à l'API
        if ( 201 == response.status_code ): # on obtient un code 201 si tout s'est bien passé
            parsed = json.loads(response.text) # on parse la réponse de l'API
            print('La donnée ' + str(num) + ' a bien été créée : ' + parsed["payload"]["id"] + '\n') # affichage d'un message de succès
            outputData[0] = parsed["payload"]["id"] # on stocke les informations nécessaire pour le fichier de sortie
            outputData[3] = 'OK'
            outputData[4] = response.text
            
            
            data_pd.iloc[num,11] = 1
            data_pd.iloc[num,12] = str(datetime.now())[0:19]
            data_pd.iloc[num,13] = link_base + parsed["payload"]["id"]
            
            print(data_pd.iloc[num])
            data_pd.to_csv('photos_info_nkl.csv', index=False)
            
        else:
            print("Une erreur {} s'est produite !".format(response.status_code))
            json_resp = json.loads(response.text)
            print(json_resp)
            #print(json_resp['message'], json_resp['payload'], sep=' | ') # en cas d'erreur, on affiche un message
            print('\n')
            outputData[3] = 'ERROR' # on complète les informations nécessaires pour le fichier de sortie
            outputData[4] = response.text
            
            data_pd.iloc[num,12] = str(datetime.now())[0:19]
            data_pd.iloc[num,14] = str(parsed)
            
            print(data_pd.iloc[num])
            data_pd.to_csv('photos_info_nkl.csv', index=False)          
            
            
        outputWriter.writerow(outputData) # on écrit les informations dans le fichier de sortie
        
        
        
    else:
        print("{} already uploaded".format(title))

# 4. Fermeture du fichier de sortie
output.close()

QD-2009-IMG2643 already uploaded
/// ['Quentin Devers', 'Upper Ladakh', 'Nyarma', 'Temple Ruin', 't045']
CREATION DE LA DONNEE 1 : QD-2009-IMG2644
Envoi du fichier base photos/photos terrains/Ladakh Upper/Nyarma - temple complex/Nyarma - temple 1 t045/QD-2009-IMG2644.JPG...
La donnée 1 a bien été créée : 10.34847/nkl.727836o6

Status                                                    pending
Title                                             QD-2009-IMG2644
Path            base photos/photos terrains/Ladakh Upper/Nyarm...
Description     This is a picture of a heritage site in Ladakh...
Creator                        Devers,Quentin,0000-0001-8469-0165
Year                                                         2009
Keywords        Quentin Devers,Upper Ladakh,Nyarma,Temple Ruin...
Type                                                        Image
License                                              CC-BY-NC-4.0
Collections                                      Temple Ruin,t045
Date scanne

KeyboardInterrupt: 